#Model definition


In [ ]:
import numpy as np
import torch
import torch.nn as nn

In [ ]:
!git clone https://github.com/PepeCamposGarcia/PINNs.git

The same fluid and physical phenomena as the direct case, but in this one, the aim of the model is to infer the parameter nu and rho that describe the fluid from the flow fields we know.
$$
    \begin{cases}
    \frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} + v \frac{\partial u}{\partial y} = -\frac{1}{\rho}\frac{\partial p}{\partial x} + \nu \left( \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} \right), \\
     \frac{\partial v}{\partial t} + u \frac{\partial v}{\partial x} + v \frac{\partial v}{\partial y}  = -\frac{1}{\rho}\frac{\partial p}{\partial y} + \nu \left( \frac{\partial^2 v}{\partial x^2} + \frac{\partial^2 v}{\partial y^2} \right), \\
     \frac{\partial u}{\partial x} + \frac{\partial v}{\partial y} = 0,
    \end{cases}
$$

In [ ]:
class SinusoidalActivation(nn.Module):
  def forward(self,x):
    return torch.sin(2 * np.pi * x)

Arbitrary initialization of parameters.

Real values are nu = 0.01 and rho = 1

In [ ]:
nu = 0.005
rho = 0.1

In [ ]:

class NavierStokesModel(nn.Module):

############################### NET ARCHITECTURE ###############################

  def __init__(self, num_neurons, num_layers):

    super(NavierStokesModel, self).__init__()
    self.num_neurons = num_neurons
    self.num_layers = num_layers

    # Inclusion of nu and rho in model set of parameters

    self.nu = torch.tensor([nu], dtype=torch.float32, requires_grad = True)
    self.rho = torch.tensor([rho], dtype=torch.float32, requires_grad = True)

    self.nu = nn.Parameter(self.nu)
    self.rho = nn.Parameter(self.rho)

    self.register_parameter('nu', self.nu)
    self.register_parameter('rho', self.rho)


### Imput layer, fully connected to the first hidden layer, sinusoidal activation
### function for the dismiss of local minima

    layer_list = [nn.Linear(3, self.num_neurons)]
    layer_list.append(SinusoidalActivation())

### Loop for the description of the hidden layers, fully connected layers
### hiperbolic tangent activation function

    for _ in range (self.num_layers - 2):
      layer_list.append(nn.Linear(self.num_neurons, self.num_neurons))
      layer_list.append(nn.Tanh())

### Output layer, 3 outputs (u,v,p)

    layer_list.append(nn.Linear(self.num_neurons, 3))
    self.layers = nn.ModuleList(layer_list)



################## FEED-FORWARD PROPAGATION, OUTPUTS ###########################

  def forward(self, x, y, t):
     """
        Params:
        x - array of shape (N, 1), input x coordinates
        y - array of shape (N, 1), input y coordinates
        t - array of shape (N, 1), input time coordinate
        Returns:
        u - tensor of shape (N, 1), output x-velocity
        v - tensor of shape (N, 1), output y-velocity
        p - tensor of shape (N, 1), output pressure
        f - x-momentum PDE evaluation of shape (N, 1)
        g - y-momentum PDE evaluation of shape (N, 1)
        h - continuity PDE evaluation of shape (N, 1)

     """
     x = torch.tensor(x, dtype=torch.float32, requires_grad=True)
     y = torch.tensor(y, dtype=torch.float32, requires_grad=True)
     t = torch.tensor(t, dtype=torch.float32, requires_grad=True)

     input_data = torch.hstack([x, y, t])
     self.N = input_data.shape[0]

     out = input_data

     for layer in self.layers:
      out = layer(out)

     u, v, p = out[:, [0]], out[:, [1]], out[:, [2]] # (N, 1) each

     u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]
     u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
     u_y = torch.autograd.grad(u, y, grad_outputs=torch.ones_like(u), create_graph=True)[0]
     u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
     u_yy = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u_y), create_graph=True)[0]

     v_t = torch.autograd.grad(v, t, grad_outputs=torch.ones_like(v), create_graph=True)[0]
     v_x = torch.autograd.grad(v, x, grad_outputs=torch.ones_like(v), create_graph=True)[0]
     v_y = torch.autograd.grad(v, y, grad_outputs=torch.ones_like(v), create_graph=True)[0]
     v_xx = torch.autograd.grad(v_x, x, grad_outputs=torch.ones_like(v_x), create_graph=True)[0]
     v_yy = torch.autograd.grad(v_y, y, grad_outputs=torch.ones_like(v_y), create_graph=True)[0]

     p_x = torch.autograd.grad(p, x, grad_outputs=torch.ones_like(p), create_graph=True)[0]
     p_y = torch.autograd.grad(p, y, grad_outputs=torch.ones_like(p), create_graph=True)[0]

        # Evaluate momentum PDE's

     f = u_t + u * u_x + v * u_y + (1 / self.rho) * p_x - self.nu * (u_xx + u_yy) # (N, 1)
     g = v_t + u * v_x + v * v_y + (1 / self.rho) * p_y - self.nu * (v_xx + v_yy) # (N, 1)

        # Evaluate continuity PDE

     h = u_x + v_y

     return u, v, p, f, g, h

In [ ]:
##################### LOSS FUNCTION DESIGN #####################################

class NavierStokesLoss(nn.Module):
  def __init__(self):
    super().__init__()
    self.mse = nn.MSELoss()

  def forward(self, u, u_net, v, v_net, p, p_net, f, g, h):
     u = torch.tensor(u, dtype=torch.float32)
     v = torch.tensor(v, dtype=torch.float32)
     p = torch.tensor(p, dtype=torch.float32)


     L_data = self.mse(u, u_net) + self.mse(v, v_net) + self.mse(p, p_net)
     L_pde  = self.mse(f, torch.zeros_like(f)) + self.mse(g, torch.zeros_like(g)) + self.mse(h, torch.zeros_like(h))

     return L_data + L_pde

#Data loading

In [ ]:
import scipy

In [ ]:
def data_loading ():

  data = scipy.io.loadmat('/content/PINNs/Models_Data_Figures/Data/cylinder_wake.mat')
  U_star = data['U_star']  # N x 2 x T   5000 x 2 x 200
  P_star = data['p_star']  # N x T       5000 x 200
  t_star = data['t']       # T x 1       200  x 1
  X_star = data['X_star']  # N x 2       5000 x 2

  N, T= X_star.shape[0], t_star.shape[0]

### Dimension transformation into N x T

  XX = np.tile(X_star[:, [0]], (1, T))
  YY = np.tile(X_star[:, [1]], (1, T))
  TT = np.tile(t_star, (1, N)).T
  UU = U_star[:, 0, :]
  VV = U_star[:, 1, :]
  PP = P_star

### Column vector transformation dim NT x 1

  x_in = XX.flatten().reshape(-1, 1)  # NT x 1
  y_in = YY.flatten().reshape(-1, 1)  # NT x 1
  t_in = TT.flatten().reshape(-1, 1)  # NT x 1
  u_in = UU.flatten().reshape(-1, 1)  # NT x 1
  v_in = VV.flatten().reshape(-1, 1) # NT x 1
  p_in = PP.flatten().reshape(-1, 1) # NT x 1

  return x_in, y_in, t_in, u_in, v_in, p_in, (N, T)


In [ ]:
x_in, y_in, t_in, u_in, v_in, p_in, (N, T) = data_loading()

In [ ]:
from tqdm import tqdm

In [ ]:
def boundary_indices(N, T):
    """
    Returns a boolean mask marking the boundary condition for all timesteps
    Params:
    N - # of data samples (space locations) in problem
    T - # of timesteps
    Return:
    nd-array of shape (N*T, )
    """
    # Create grid for one timestep
    # Data is a 100 x 50 matriz for each timestep, each timestep constains
    # 296 boundary points out of the 5000 total, this function creates a boolean mask
    # to identify them

    grid_t_0 = np.zeros((50, 100))

    # Set boundary to 1
    grid_t_0[0, :] = 1
    grid_t_0[:, 0] = 1
    grid_t_0[-1, :] = 1
    grid_t_0[:, -1] = 1

    # np.count_nonzero(grid_t_0 == 1) = 296
    # Flatten grid into column vector and propagate for each timestep

    grid_t_in = np.tile(grid_t_0.reshape(-1, 1), (1, T)) # (N, T)


    # Flatten final grid into column vector to be used in training
    boundary_positions = grid_t_in.astype(bool).flatten() # (NT,1)

    return boundary_positions


#Training algorithm

In [ ]:
import csv

In [ ]:
def main(num_neurons, num_layers, epochs, train_selection):
    """
    Params:
    num_neurons - int, # of hidden units for each neural network layer
    num_layers - int, # of neural network layers
    epochs - int, # of training epochs
    model - int, whether to use model 1 (Raissi 2019) or model 2 (continuity PDE)
    train_selection - float, frac of all data (N*T) to select for training OR
                      'BC', select the boundary conditions for training (all timesteps)
    """
    # Load flattened cynlinder wake data
    x_in, y_in, t_in, u_in, v_in, p_in, (N, T) = data_loading () # (NT, 1)


    # Select training based on a fraction of an existing database or in boundary conditions

    if train_selection == 'BC':
        idx = boundary_indices(N, T)
    else:
        samples = int(round(N * T * train_selection))
        np.random.seed(0)
        idx = np.random.choice(x_in.shape[0], samples, replace=False)

    x_train = x_in[idx, :]
    y_train = y_in[idx, :]
    t_train = t_in[idx, :]
    u_train = u_in[idx, :]
    v_train = v_in[idx, :]
    p_train = p_in[idx, :]


    torch.manual_seed(0)

    PINN_model = NavierStokesModel(num_neurons=num_neurons, num_layers=num_layers)
    criterion = NavierStokesLoss()

    optimizer = torch.optim.LBFGS(PINN_model.parameters(), line_search_fn='strong_wolfe')

    def closure():
        """Define closure function to use with LBFGS optimizer"""
        optimizer.zero_grad()   # Clear gradients from previous iteration

        u_net, v_net, p_net, f, g, h = PINN_model(x_train, y_train, t_train)
        loss = criterion(u_train, u_net, v_train, v_net, p_train, p_net, f, g, h)

        loss.backward() # Backprogation

        return loss

    def training_loop(epochs):
        """Run full training loop"""

        with open('parameters.csv', 'w', newline='') as csvfile:

          writer = csv.writer(csvfile)
          writer.writerow(['Epoch', 'Loss','nu','rho'])

          for i in tqdm(range(epochs), desc='Training epochs: '):

            optimizer.step(closure)
            loss = closure().item()
            writer.writerow([i + 1, loss, PINN_model.nu.item(), PINN_model.rho.item()])



    training_loop(epochs=epochs)

    torch.save(PINN_model.state_dict(), 'model_name.pth')
    return

Running the following cell will start the training process. This will produce 2 files: 'model_name.pth', which contains the trained parameters of the model for its evaluation, and 'parameters.csv' which contains the evolution of the predicted parameters along the iterations. The number of neurons, layers, epochs and the training selection fraction can be altered. Initialization of the parameters plays a fundamental role in the performance of the model, this can also be changed at the beginning of the notebook.

In [ ]:
if __name__ == '__main__':
    main(num_neurons=30, num_layers=6, epochs=500, train_selection=0.005)

Training epochs:   0%|          | 1/200 [00:02<09:54,  2.99s/it]

𝜆_real = [0.01,  1], 𝜆_PINN = [0.71948,  2.01470]


Training epochs:   6%|▌         | 11/200 [00:32<09:27,  3.00s/it]

𝜆_real = [0.01,  1], 𝜆_PINN = [-0.00180,  1.35258]


Training epochs:  10%|█         | 21/200 [01:00<08:24,  2.82s/it]

...

Training epochs:  90%|█████████ | 181/200 [09:24<01:03,  3.35s/it]

𝜆_real = [0.01,  1], 𝜆_PINN = [0.01020,  1.02470]


Training epochs:  96%|█████████▌| 191/200 [09:56<00:27,  3.04s/it]

𝜆_real = [0.01,  1], 𝜆_PINN = [0.01018,  1.02497]


Training epochs: 100%|██████████| 200/200 [10:24<00:00,  3.12s/it]


This is an example for the loading and consultation of the parameters' value predicted. In this case the initial values for nu and rho were 0.005 and 0.1 respectively. Every model in this directory was trained on 0.5% of the total data, with 6 layers and 30 neurons per layer.

In [ ]:
def load_saved_model(num_layers, num_neurons):
    """
    Params:
    num_neurons - int, # of hidden units for each neural network layer
    num_layers - int, # of neural network layers
    """
    # load saved state_dict
    PINN_model = NavierStokesModel(num_neurons=num_neurons, num_layers=num_layers)
    PINN_model.load_state_dict(torch.load('/content/PINNs/Models_Data_Figures/NAVIERSTOKES/Inverse/Models/NSInverse_0005_01.pth'))
    PINN_model.eval() # Set model to evaluation mode
    return PINN_model

In [ ]:
modelnavierstokes = load_saved_model(num_layers=6, num_neurons=30)

In [ ]:
modelnavierstokes.rho.detach().numpy()

array([1.0021018], dtype=float32)

In [ ]:
modelnavierstokes.nu.detach().numpy()

array([0.01073081], dtype=float32)